In [1]:
# Date and Market added FINAL and data scraping is done for N days

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
    
f = int(input('Enter upto How many days You want to scrap data from Today: '))
dt_lst = []
today = datetime.date.today()
for d in range(2,f):
    start_date = today - datetime.timedelta(d)
    start_date_string = str(start_date.strftime("%d-%m-%Y"))
    dt_lst.append(start_date_string)


#p_1 = []
#for n in range(0,100):
#    p_1.append('//*[@id="ContentPlaceHolder1_DataList1_Button1_' + str(n) +'"]')

p_2 = []
for m in range(0,5):
    p_2.append('//*[@id="ContentPlaceHolder1_grdDaily"]/tbody/tr[' + str(m) +']/td[2]/a')

df = pd.DataFrame()
df_list = []
try:
    for dt in range(len(dt_lst)):
        print(dt_lst[dt])
        date_td = dt_lst[dt]
        p_1 = []
        for n in range(0, 100):
            p_1.append('//*[@id="ContentPlaceHolder1_DataList1_Button1_' + str(n) + '"]')
        for w in range(0,110):
            print(f'w value after: {w}')
            try:
                PATH = "C:\Program Files (x86)\chromedriver.exe"
                options = Options()
                options.add_argument("--headless")
                driver = webdriver.Chrome(chrome_options=options, executable_path=PATH)
                driver.get('http://tsmarketing.in/HomePageGe.aspx')


                dateclick = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
                dateclick.click()
                dateclick.send_keys(Keys.CONTROL, 'a')
                #dateclick.send_keys(Keys.BACK_SPACE)
                dateclick.send_keys(dt_lst[dt])
                dateclick.send_keys(Keys.RETURN)

                for p1 in p_1:
                    search = driver.find_element_by_xpath(p1)
                    print(search.text)
                    p_1.remove(p1)
                    search.send_keys(Keys.RETURN)
                    for p2 in p_2:
                        try:
                            link_text = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, p2)))
                            print(link_text.text)
                            mar = link_text.text
                            link_text.send_keys(Keys.RETURN)

                            soup_level3 = BeautifulSoup(driver.page_source, 'lxml')
                            table = soup_level3.find("table",{"id":"ContentPlaceHolder1_grdDaily"})
                            table_rows = table.find_all('tr')

                            test_list = []
                            for tr in table_rows:
                                td = tr.find_all('td')
                                row = [tr.text for tr in td]
                                test_list.append(row)

                            column_list = []
                            for tr in table_rows:
                                th = tr.find_all('th')
                                col = [tr.text for tr in th]
                                column_list.append(col)


                            test_df = pd.DataFrame(test_list)
                            test_df.columns = column_list[0]
                            test_df = test_df.apply(lambda line: line.str.strip().replace('\n', ''))
                            df = test_df
                            df['Market_Name'] = mar
                            df['Date'] = date_td
                            df_list.append(df)
                            print(len(df_list))
                            driver.back() # back from page 3 to 2 for all three market

                        except Exception as e:
                            #print(e)
                            driver.quit()


            except:
                pass
except Exception as e:
    print(e)
#print(len(df_list))

Enter upto How many days You want to scrap data from Today: 4
07-06-2021


C:\Users\Sanjeeb\anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: use options instead of chrome_options


Groundnut pods
Bowenpally
1
Onions
Bowenpally
2
Gudimalkapur Vegetable Market
3
MMC
4
Tamarind
MMC
5
Apple
Gaddiannaram
6
Cherry
Gaddiannaram
7
Dragon Fruit
Gaddiannaram
8
Grapes
Gaddiannaram
9
Grapes
Gaddiannaram
10
Guava
Gaddiannaram
11
Guava
Gaddiannaram
12
Jamun
Gaddiannaram
13
Kiwi
Gaddiannaram
14
Mango
Gaddiannaram
15
Mango
Gaddiannaram
16
Mango
Gaddiannaram
17
Mosambi
Gaddiannaram
18
Nashpati(Pear)
Gaddiannaram
19
Orange
Gaddiannaram
20
Papaya
Gaddiannaram
21
Papaya
Gaddiannaram
22
Pine apple
Gaddiannaram
23
Plum(Aloobukara)
Gaddiannaram
24
Pomegrenates
Gaddiannaram
25
Sapota
Gaddiannaram
26
Water Melon
Gaddiannaram
27
Arati-Green Plantain
Gudimalkapur Vegetable Market
28
LB Nagar
29
Arati-Green Plantain
Gudimalkapur Vegetable Market
30
LB Nagar
31
Beet Root
Bowenpally
32
Gudimalkapur Vegetable Market
33
Bitter Gourd
Bowenpally
34
Gudimalkapur Vegetable Market
35
Bitter Gourd
Bowenpally
36
Gudimalkapur Vegetable Market
37
Bottle Gourd
Bowenpally
38
Gudimalkapur Vegetable Market


In [2]:
#print(len(df_list))
list_f = []
for t in range(len(df_list)):
    temp = "f" + str(t)
    list_f.append(temp)
    

final_df = df_list[0]
for u in range(1,len(df_list)):
    l = list_f[u]
    l = df_list[u]
    final_df = pd.concat([final_df,l])
#print(final_df)
try:
    final_df.drop(['Market Code', 'Market'], axis=1, inplace=True)
except:
    pass
final_df.dropna(inplace=True)
final_df = final_df[final_df['Commission Agent'] != '']
final_df.to_csv(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\generatedCSV\d07_06_21_notMapped.csv", index=False)

## Mapping Mandal and Village

In [3]:
#final_df = pd.read_csv(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\generatedCSV\d13_05_21_notMapped.csv")
final_df['Mandal'] = final_df['Mandal'].str.upper()
final_df['Village'] = final_df['Village'].str.upper()
final_df['Commodity'] = final_df['Commodity'].str.upper()
final_df['Vehicle No'] = final_df['Vehicle No'].str.upper()
final_df['Market_Name'] = final_df['Market_Name'].str.upper()
#final_df.head()

In [4]:
final_df.columns

Index(['No.', 'Commission Agent', 'Farmer Name', 'Quantity', 'Units', 'Mandal',
       'Village', 'Vehicle', 'Vehicle No', 'Commodity', 'Lot Number',
       'Market_Name'],
      dtype='object')

In [5]:
final_df.columns = ['No', 'Commission_Agent', 'Farmer_Name', 'Quantity', 'Units', 'Mandal','Village', 'Vehicle', 'Vehicle_No', 'Commodity', 'Lot_Number', 'Market_Name', 'Date']

In [6]:
df_district = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=1, usecols=['name','loclevel3','loclevel2'])
df_district['name'] = df_district['name'].str.upper()
#df_district.head()

dict_3_to_2 = dict(zip(df_district['loclevel3'].tolist(), df_district['loclevel2'].tolist()))
#dict_3_to_2
dict_3_to_district = dict(zip(df_district['loclevel3'].tolist(), df_district['name'].tolist()))
#dict_3_to_district
dict_district_to_3 = dict(zip(df_district['name'].tolist(), df_district['loclevel3'].tolist()))
#dict_district_to_3

In [7]:
df_dist_to_dist = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=7)
df_dist_to_dist['name1'] = df_dist_to_dist['name1'].str.upper()
df_dist_to_dist['name2'] = df_dist_to_dist['name2'].str.upper()
#df_dist_to_dist.head()

dict_dist_to_dist = dict(zip(df_dist_to_dist['name1'].tolist(), df_dist_to_dist['name2'].tolist()))
#print(dict_dist_to_dist)

In [8]:
df_state = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=2)
df_state['name'] = df_state['name'].str.upper()
#df_state

dict_id_to_state = dict(zip(df_state['ID'].tolist(), df_state['name'].tolist()))
#print(dict_id_to_state)
dict_id_to_shortname = dict(zip(df_state['ID'].tolist(), df_state['shortName'].tolist()))
#print(dict_id_to_shortname)
dict_shortname_to_state = dict(zip(df_state['shortName'].tolist(), df_state['name'].tolist()))
#print(dict_shortname_to_state)

In [9]:
df_state_to_state = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=6)
df_state_to_state['name1'] = df_state_to_state['name1'].str.upper()
df_state_to_state['name2'] = df_state_to_state['name2'].str.upper()
#df_state_to_state.head()

dict_state_to_state = dict(zip(df_state_to_state['name1'].tolist(), df_state_to_state['name2'].tolist()))
#print(dict_state_to_state)

In [10]:
df_mandal = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=4, usecols=['name', 'loclevel3'])
df_mandal['name'] = df_mandal['name'].str.upper()
#print(df_mandal.head())

dict_mandal_to_3 = dict(zip(df_mandal['name'].tolist(), df_mandal['loclevel3'].tolist()))
#print(dict_mandal_to_3)

In [11]:
final_df['DistrictID_M'] = final_df.Mandal.map(dict_mandal_to_3)
final_df['DistrictM'] = final_df.DistrictID_M.map(dict_3_to_district)
final_df['ID2_3'] = final_df.DistrictID_M.map(dict_3_to_2)
final_df['StateM'] = final_df.ID2_3.map(dict_id_to_state)
#final_df['State_short_nameM'] = final_df.ID2_3.map(dict_id_to_shortname)

final_df.drop(['DistrictID_M', 'ID2_3'], axis=1, inplace=True)
#final_df.head()

In [12]:
final_df['DistrictID_V'] = final_df.Village.map(dict_mandal_to_3)
final_df['DistrictV'] = final_df.DistrictID_V.map(dict_3_to_district)
final_df['ID2_3_V'] = final_df.DistrictID_V.map(dict_3_to_2)
final_df['StateV'] = final_df.ID2_3_V.map(dict_id_to_state)
#final_df['State_short_nameV'] = final_df.ID2_3_V.map(dict_id_to_shortname)

final_df.drop(['DistrictID_V', 'ID2_3_V'], axis=1, inplace=True)
#final_df.tail(20)

In [13]:
final_df['State1'] = final_df.Village.map(dict_shortname_to_state)
final_df['State2'] = final_df.Village.map(dict_state_to_state)
final_df['District1'] = final_df.Village.map(dict_dist_to_dist)
final_df['level3'] = final_df.District1.map(dict_district_to_3)
final_df['level2'] = final_df.level3.map(dict_3_to_2)
final_df['State3'] = final_df.level2.map(dict_id_to_state)

final_df.drop(['level3', 'level2'], axis=1, inplace=True)


In [14]:
final_df.tail(20)

,No,Commission_Agent,Farmer_Name,Quantity,Units,Mandal,Village,Vehicle,Vehicle_No,Commodity,Lot_Number,Market_Name,DistrictM,StateM,DistrictV,StateV,State1,State2,District1,State3
8,8,"31N,Devara Vamshi F",JILANI,50,KGs,,C.B.PUR,2,KA40A 4533,TIGER/RUBY,A77621-263,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,"28N,Jyothi F.M",JILANI,30,KGs,,C.B.PUR,2,KA40A 4533,TIGER/RUBY,A77621-258,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,"26N,Dasanjaneya F.M",JILANI,100,KGs,,C.B.PUR,2,KA40A 4533,TIGER/RUBY,A77621-257,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,"78N,Taj FM",JILANI,50,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-193,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,12,"75N,Durga FM",JILANI,60,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-191,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,13,"74N,Kaliyuga Venkateshwara FM",JILANI,50,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-189,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,"72N,Sai Yadav FM",JILANI,100,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-187,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,"96,B.Siddartha Reddy F.M",JILANI,50,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-185,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,16,"89,Jagan F.M",JILANI,50,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-184,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,17,"87,Ome Shiva F.M",JILANI,50,KGs,,C.B.PUR,2,KA40 9359,TIGER/RUBY,A77621-181,GUDIMALKAPUR FLOWER MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Truck Number Mapping

In [15]:
df_truck_no = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=9, usecols=['Code', 'Jurisdiction', 'State'])
df_truck_no['Code'] = df_truck_no['Code'].str.upper()
#print(df_truck_no.head())
#final_df['Vehicle No'].head()

In [16]:
df_truck_no['Code'] = df_truck_no['Code'].str.replace( "[-]", "")
final_df['Vehicle_No'] = final_df['Vehicle_No'].apply(lambda x:x[0:4])

#print(final_df['Vehicle No'].head())
#print(df_truck_no['Code'].head())

C:\Users\Sanjeeb\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [17]:
dict_code_to_juri = dict(zip(df_truck_no['Code'].tolist(), df_truck_no['Jurisdiction'].tolist()))
#print(dict_code_to_juri)
dict_code_to_state = dict(zip(df_truck_no['Code'].tolist(), df_truck_no['State'].tolist()))
#print(dict_code_to_state)

In [18]:
final_df['Vehicle_Jurisdiction'] = final_df.Vehicle_No.map(dict_code_to_juri)
final_df['Vehicle_State'] = final_df.Vehicle_No.map(dict_code_to_state)

In [19]:
final_df.head()

,No,Commission_Agent,Farmer_Name,Quantity,Units,Mandal,Village,Vehicle,Vehicle_No,Commodity,...,DistrictM,StateM,DistrictV,StateV,State1,State2,District1,State3,Vehicle_Jurisdiction,Vehicle_State
1,1,"A28/W, R.Srinivas",YADAGIRI,8,Bags,TUPRAN,TPN,5,TS35,GROUNDNUT PODS,...,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Medak district,Telangana
2,2,"B13, Terala Bhasker Rao",CHARY,30,Bags,JANGAON,JANGAIN,5,TS27,GROUNDNUT PODS,...,WARANGAL,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Jangaon district,Telangana
3,3,"B13/2/E, Maruthi VCA",MAHESH,20,Bags,GAJWEL,GAJWEL,5,TS07,GROUNDNUT PODS,...,MEDAK,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Ranga Reddy,Telangana
4,4,"B13/2/E, Maruthi VCA",SWAMY,10,Bags,GAJWEL,GAJWEL,5,TS36,GROUNDNUT PODS,...,MEDAK,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Siddipet district,Telangana
5,5,"B13, Terala Bhasker Rao",SRIDER,28,Bags,JANGAON,JANGAON,7,TS03,GROUNDNUT PODS,...,WARANGAL,TELANGANA,WARANGAL,TELANGANA,NaN,NaN,JANGAON,TELANGANA,Warangal Urban district,Telangana


# Mapping Commodity

In [20]:
df_commodity = pd.read_excel(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\Gramoday_Datasets_market.xlsx", sheet_name=8, usecols=['Commodity', 'Changes', 'Category'])
df_commodity['Commodity'] = df_commodity['Commodity'].str.upper()
df_commodity['Changes'] = df_commodity['Changes'].str.upper()
df_commodity['Category'] = df_commodity['Category'].str.upper()
#print(df_commodity.head(20))

dict_commodity_to_changes = dict(zip(df_commodity['Commodity'].tolist(), df_commodity['Changes'].tolist()))
#print(dict_commodity_to_changes)
dict_changes_to_category = dict(zip(df_commodity['Changes'].tolist(), df_commodity['Category'].tolist()))
#print(dict_changes_to_category)

In [21]:
final_df['Commodity'] = final_df.Commodity.map(dict_commodity_to_changes)
final_df['Category'] = final_df.Commodity.map(dict_changes_to_category)

In [22]:
final_df.head(10)

,No,Commission_Agent,Farmer_Name,Quantity,Units,Mandal,Village,Vehicle,Vehicle_No,Commodity,...,StateM,DistrictV,StateV,State1,State2,District1,State3,Vehicle_Jurisdiction,Vehicle_State,Category
1,1,"A28/W, R.Srinivas",YADAGIRI,8,Bags,TUPRAN,TPN,5,TS35,GROUNDNUT,...,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Medak district,Telangana,OILSEEDS
2,2,"B13, Terala Bhasker Rao",CHARY,30,Bags,JANGAON,JANGAIN,5,TS27,GROUNDNUT,...,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Jangaon district,Telangana,OILSEEDS
3,3,"B13/2/E, Maruthi VCA",MAHESH,20,Bags,GAJWEL,GAJWEL,5,TS07,GROUNDNUT,...,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Ranga Reddy,Telangana,OILSEEDS
4,4,"B13/2/E, Maruthi VCA",SWAMY,10,Bags,GAJWEL,GAJWEL,5,TS36,GROUNDNUT,...,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Siddipet district,Telangana,OILSEEDS
5,5,"B13, Terala Bhasker Rao",SRIDER,28,Bags,JANGAON,JANGAON,7,TS03,GROUNDNUT,...,TELANGANA,WARANGAL,TELANGANA,NaN,NaN,JANGAON,TELANGANA,Warangal Urban district,Telangana,OILSEEDS
6,6,"C13, Mohd. Ali",SRIDER,5,Bags,JANGAON,JANGAON,7,TS03,GROUNDNUT,...,TELANGANA,WARANGAL,TELANGANA,NaN,NaN,JANGAON,TELANGANA,Warangal Urban district,Telangana,OILSEEDS
7,7,"B13/2/E, Maruthi VCA",GOPAL,15,Bags,TUPRAN,TUPRAN,5,TS36,GROUNDNUT,...,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Siddipet district,Telangana,OILSEEDS
8,8,"A28/W, R.Srinivas",GOPAL,40,Bags,TUPRAN,TUPRAN,5,TS36,GROUNDNUT,...,TELANGANA,MEDAK,TELANGANA,NaN,NaN,NaN,NaN,Siddipet district,Telangana,OILSEEDS
9,9,"B11/1/W, Sri Mahankali Veg",NARESH,7,Bags,SIDDIPET,DHANDUPALLE,7,TS35,GROUNDNUT,...,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Medak district,Telangana,OILSEEDS
10,10,Sri Laxmi Narasimha Veg,NARESH,10,Bags,SIDDIPET,DHANDUPALLE,7,TS35,GROUNDNUT,...,TELANGANA,NaN,NaN,NaN,NaN,NaN,NaN,Medak district,Telangana,OILSEEDS


In [23]:
final_df.columns

Index(['No', 'Commission_Agent', 'Farmer_Name', 'Quantity', 'Units', 'Mandal',
       'Village', 'Vehicle', 'Vehicle_No', 'Commodity', 'Lot_Number',
       'Market_Name', 'DistrictM', 'StateM', 'DistrictV', 'StateV', 'State1',
       'State2', 'District1', 'State3', 'Vehicle_Jurisdiction',
       'Vehicle_State', 'Category'],
      dtype='object')

In [24]:
final_df.to_csv(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\generatedCSV\d07_06_21_Mapped.csv", index=False)

# Superimposing Columns

In [25]:
df = final_df[['StateM', 'StateV', 'State1', 'State2', 'State3', 'DistrictM', 'DistrictV', 'District1']]
#df = pd.read_csv(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\generatedCSV\d07_06_21_Mapped.csv", usecols=['StateM', 'StateV', 'State1', 'State2', 'State3', 'DistrictM', 'DistrictV', 'District1'])
df.replace(to_replace=np.nan, value='', inplace=True)
df.columns = ['DistrictA', 'StateA', 'DistrictB', 'StateB', 'StateC', 'StateD','DistrictC', 'StateE']
#print(df.head(10))

lst_index = []
lst_state = []
for index, row in df.iterrows():
    lst_state.append(row['StateA'] or row['StateB'] or row['StateC'] or row['StateD'] or row['StateE'])
    lst_index.append(index)
#print(len(lst_state))
df_state = pd.DataFrame()
df_state["State"] = lst_state
df_state["IndexNo"] = lst_index
#print(df_state.head(10))

lst_district = []
for index, row in df.iterrows():
    lst_district.append(row['DistrictA'] or row['DistrictB'] or row['DistrictC'])
#print(len(lst_district))
df_district = pd.DataFrame()
df_district["District"] = lst_district
df_district["IndexNo"] = lst_index
#print(df_district.head(10))

df["IndexNo"] = lst_index
final_df["IndexNo"] = lst_index
#print(df.head(10))

t1 = pd.merge(df_state,df_district,on='IndexNo')
#t2 = pd.merge(df,t1,on='IndexNo')
final_dataframe = pd.merge(final_df,t1,on='IndexNo')
final_dataframe.drop(['DistrictM', 'StateM', 'DistrictV', 'StateV', 'State1','State2', 'District1', 'State3', 'IndexNo'], axis=1,inplace=True)
#final_dataframe.head(10)

In [27]:
final_dataframe.to_csv(r"C:\Users\Sanjeeb\Desktop\Gramoday\2webScrap\generatedCSV\d07_06_21_clean.csv", index=False)